# Image_Proccess_Resize

### Import scripts

In [1]:
%matplotlib inline
from script.util import *
from script.load_data_gray import *

In [2]:
t0 = time.time()

run_name = get_run_name('SC_Iceberg_Classifier', 'Image_Proccess_Resize')
html_folder = get_html_folder(True)
input_folder = get_input_folder(True)
input_processed_folder = get_input_processed_folder(True)
log_folder = get_log_folder(True)
model_folder = get_model_folder(True)
output_folder = get_output_folder(True)

run_name: SC_Iceberg_Classifier-Image_Proccess_Resize-20180112_215345
D:\Kaggle\statoil-iceberg-classifier-challenge\html
D:\Kaggle\statoil-iceberg-classifier-challenge\input
D:\Kaggle\statoil-iceberg-classifier-challenge\input\processed
D:\Kaggle\statoil-iceberg-classifier-challenge\log
D:\Kaggle\statoil-iceberg-classifier-challenge\model
D:\Kaggle\statoil-iceberg-classifier-challenge\output


In [3]:
%%time
id_data, id_test = load_id()
y_data = load_y_data()
band1_data, band2_data, band1_test, band2_test = load_band_data()
x_data, x_test = load_data()

(1604,) 000d71b1 fffe76e4 12928
(8424,) 000c65d7 fff77564 67488
(1604,) 0 1 12928
(1604, 75, 75) -45.594448 34.574917 72180128
(1604, 75, 75) -45.655499 20.154249 72180128
(8424, 75, 75) -45.6805458069 37.090523 379080128
(8424, 75, 75) -49.0835 37.0905227661 379080128
(1604, 75, 75, 3) -45.655499 34.574917 216540144
(8424, 75, 75, 3) -49.0835 37.090523 1137240144
Wall time: 5.88 s


In [4]:
from skimage import transform, data

def resize_image(arr, size=150):
    new_arr = np.zeros((arr.shape[0], size, size))
    for i, img in enumerate(arr):
        new_arr[i] = transform.resize(img, (size, size), mode='reflect')
    return new_arr
# test
aa = np.ones((3, 75, 75))
bb = resize_image(aa)
print(bb.shape)

(3, 150, 150)


### Resize

In [5]:
target_size = 150

In [6]:
%%time
band1_data_resize = resize_image(band1_data, target_size)
band2_data_resize = resize_image(band2_data, target_size)
band1_test_resize = resize_image(band1_test, target_size)
band2_test_resize = resize_image(band2_test, target_size)

band1_data_resize_file = os.path.join(get_input_processed_folder(), 'band1_data%s.p' % target_size)
band2_data_resize_file = os.path.join(get_input_processed_folder(), 'band2_data%s.p' % target_size)
band1_test_resize_file = os.path.join(get_input_processed_folder(), 'band1_test%s.p' % target_size)
band2_test_resize_file = os.path.join(get_input_processed_folder(), 'band2_test%s.p' % target_size)

np.save(band1_data_resize_file, band1_data_resize)
np.save(band2_data_resize_file, band2_data_resize)
np.save(band1_test_resize_file, band1_test_resize)
np.save(band2_test_resize_file, band2_test_resize)

Wall time: 1min 4s


In [7]:
%%time
band1_data_gray_resize = resize_image(band1_data_gray, target_size)
band2_data_gray_resize = resize_image(band2_data_gray, target_size)
band1_test_gray_resize = resize_image(band1_test_gray, target_size)
band2_test_gray_resize = resize_image(band2_test_gray, target_size)

band1_data_gray_resize_file = os.path.join(get_input_processed_folder(), 'band1_data_gray%s.p' % target_size)
band2_data_gray_resize_file = os.path.join(get_input_processed_folder(), 'band2_data_gray%s.p' % target_size)
band1_test_gray_resize_file = os.path.join(get_input_processed_folder(), 'band1_test_gray%s.p' % target_size)
band2_test_gray_resize_file = os.path.join(get_input_processed_folder(), 'band2_test_gray%s.p' % target_size)

np.save(band1_data_gray_resize_file, band1_data_gray_resize)
np.save(band2_data_gray_resize_file, band2_data_gray_resize)
np.save(band1_test_gray_resize_file, band1_test_gray_resize)
np.save(band2_test_gray_resize_file, band2_test_gray_resize)

NameError: name 'band1_data_gray' is not defined

In [8]:
band1_data_resize = np.load(band1_data_resize_file)
band2_data_resize = np.load(band2_data_resize_file)
band1_test_resize = np.load(band1_test_resize_file)
band2_test_resize = np.load(band2_test_resize_file)

band1_data_gray_resize = np.load(band1_data_gray_resize_file)
band2_data_gray_resize = np.load(band2_data_gray_resize_file)
band1_test_gray_resize = np.load(band1_test_gray_resize_file)
band2_test_gray_resize = np.load(band2_test_gray_resize_file)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Kaggle\\statoil-iceberg-classifier-challenge\\input\\processed\\band1_data150.p'

In [ ]:
describe(id_data)
describe(id_test)
describe(y_data)

describe(band1_data_resize)
describe(band2_data_resize)
describe(band1_test_resize)
describe(band2_test_resize)

describe(band1_data_gray_resize)
describe(band2_data_gray_resize)
describe(band1_test_gray_resize)
describe(band2_test_gray_resize)

In [ ]:
show_data_images(6, 3, id_data, y_data, band1_data, band1_data200, band2_data, band2_data200)
show_data_images(6, 3, id_test, None, band1_test, band1_test200, band2_test, band2_test200)

In [ ]:
show_data_images(6, 3, id_test, y_data, band1_data_gray, band1_data_gray200, band2_data_gray, band2_data_gray200)
show_data_images(6, 3, id_test, None, band1_test_gray, band1_test_gray200, band2_test_gray, band2_test_gray200)

In [ ]:
t1 = time.time()
print('time cost: %.2f s' % (t1-t0))
print('Done!')

In [ ]:
print(run_name)